In [1]:
from __future__ import print_function, division
from collections import defaultdict, Counter
from tqdm import tqdm
from aer import read_naacl_alignments, AERSufficientStatistics
from random import random
from scipy import special
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import os

## Read in the data

In [2]:
english_train = 'training/hansards.36.2.e'
french_train = 'training/hansards.36.2.f'
english_val = 'validation/dev.e'
french_val = 'validation/dev.f'
fname = 'naacltest.txt'

def add_unk(original_sentences, threshold, ext_sentences=None):
    #use original sentences and unk low frequency words
    if ext_sentences is None: 
        counts = Counter([item for sublist in original_sentences for item in sublist])
        unk_words = list({k:counts[k] for k in counts if counts[k] <= threshold})
    #use external sentences and unk unknown words
    else:
        counts = Counter([item for sublist in ext_sentences for item in sublist])
        unk_words = list({k:counts[k] for k in counts})
    for line, sentence in enumerate(tqdm(original_sentences)):
        for index, word in enumerate(sentence):
            if word in unk_words and ext_sentences is None:
                original_sentences[line][index] = '-UNK-'
            elif word not in unk_words and ext_sentences is not None:
                original_sentences[line][index] = '-UNK-'
    return original_sentences
    

def read_data(english_file, french_file, unk=False, threshold=1, ttype='training', eng_data=None, fre_data=None):
    english_sentences = []
    french_sentences = []
    with open(english_file, 'r', encoding='utf8') as engf, open(french_file, 'r', encoding='utf8') as fref:
        for line in engf:
            english_sentences.append(["NULL"] + line.split())
        for line in fref:
            french_sentences.append(line.split())
    
    #unk cases
    if unk:
        print("Adding the -UNK- token to the data.")
        englishname = ttype +'_'+ str(threshold)+'_unk.e'
        frenchname = ttype +'_'+ str(threshold)+'_unk.f'
        #load if file is found
        if os.path.isfile(englishname):
            with open (englishname, 'rb') as eng:
                english_sentences = pickle.load(eng)
        else:
            english_sentences = add_unk(english_sentences, threshold, eng_data)
            with open(englishname, 'wb') as eng:
                pickle.dump(english_sentences, eng)
        print("English data complete.")
        
        
        if os.path.isfile(frenchname):
            with open(frenchname, 'rb') as fre:
                french_sentences = pickle.load(fre)
        else:
            french_sentences = add_unk(french_sentences, threshold, fre_data)
            with open(frenchname, 'wb') as fre:
                pickle.dump(french_sentences, fre)             
        print("French data complete")          
        
    assert len(english_sentences) == len(french_sentences), 'data mismatch'
    return list(zip(english_sentences, french_sentences))

training_data = read_data(english_train, french_train, True)
ext_data = list(zip(*training_data))
validation_data = read_data(english_val, french_val, True, ttype='validation', eng_data=ext_data[0], fre_data=ext_data[1])

Adding the -UNK- token to the data.
English data complete.
French data complete
Adding the -UNK- token to the data.
English data complete.
French data complete


# AER Cell

In [3]:
def test(path, personal_sets=None):
    from random import random
    # 1. Read in gold alignments
    gold_sets = read_naacl_alignments('validation/dev.wa.nonullalign')

    # 2. Here you would have the predictions of your own algorithm
    if personal_sets is None:
        personal_sets = read_naacl_alignments(path)
        predictions = []
        for s, p in personal_sets:
            links = set()
            for link in s:
                links.add(link)
            predictions.append(links)
    else:
        predictions=personal_sets

    # 3. Compute AER
    # first we get an object that manages sufficient statistics 
    metric = AERSufficientStatistics()
    # then we iterate over the corpus 
    for gold, pred in zip(gold_sets, predictions):
        metric.update(sure=gold[0], probable=gold[1], predicted=pred)
    return metric.aer()

#hardcoded AER test for 5 iterations
for i in range(20):
    aer = test('iteration'+str(i)+'.txt')
    print(aer)

0.34416826003824097
0.5252140818268316
0.5368421052631579
0.5368421052631579
0.5363288718929254
0.5391969407265774
0.5391221374045801
0.5381679389312977
0.5381679389312977
0.5380952380952381
0.31800766283524906
0.3137065637065637
0.3137065637065637
0.31436837029893927
0.3153326904532304
0.3153326904532304
0.3153326904532304
0.3153326904532304
0.3153326904532304
0.3153326904532304


## IBM Model 1: Generative Process
Step 1: Pick an alignmet $a$ with probability  p(a|e,m) = $\frac{1}{(l+1)^m}$

Step 2: pick the French words with probability

p(f|a,e,m) = $\prod^m_{j=1} t(f_j | e_aj )$



P(f,a|e, m ) = p(a|e,m) $\times$ p(f|a,e,m)


In [4]:
def align_all(data, translate_dict, fname=None):
    """Create alignments for pairs of English and French sentences.
    Both save them as sets per sentence and pair and save to file.
    
    Args:
        validation: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f
        fname: filename to save alignments in, in NAACL format

    Returns:
        list of sets
    """
    file = open(fname, 'w')
    alignments = []
    for k, (english_words, french_words) in enumerate(data):
        alignment = align(english_words, french_words, translate_dict, False)
        for pos1, pos2 in alignment:
            file.write("{} {} {}\n".format(str(k+1), str(pos1), str(pos2)))
        alignments.append(set(alignment))
    return alignments
    
def align(english_words, french_words, translate_dict, add_null=True):
    """Align one sentence pair, either with or without the NULL alignments.
    
    Args:
        english_words: list of english words
        french_words: list of french words
        translate_dict: dictionary with translation probabilities e to f
        add_null: boolean to indicate whether NULL alignments should be included

    Return:
        list of tuples
    """
    alignment = []
    for j, fword in enumerate(french_words):
        prior = 0.0
        alignment_j = 0
        for i, eword in enumerate(english_words):
            # Only include terms that are in the dictionary
            if eword in translate_dict and fword in translate_dict[eword]:
                prob = translate_dict[eword][fword]
                if prob > prior:
                    prior = prob
                    alignment_j = i
        # Add dependent on whether it's a NULL alignments
        if alignment_j != 0 or add_null:
            alignment.append((alignment_j, j + 1))
    return alignment

def log_likelihood(data, translate_dict, add_constant=False):
    """
    Args:
        data: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f

    Returns:
        float: log likelihood
    """
    log_likelihood = 0
    for e, f in data:
        alignment = align(e, f, translate_dict, True)
        prob = 0
        for j, i in alignment:
            prob += math.log(translate_dict[e[j]][f[i-1]])
        log_likelihood += prob
        if add_constant:
            log_likelihood += -len(f) * np.log(len(e) + 1)
    return log_likelihood

def initialize_t(data, uniform=True):
    """Initialise the translation probabilities.
    
    Args:
        data: list of tuples, english and french sentences
        uniform: boolean indicating initialisation type

    Returns:
        defaultdict(Counter)
    """
    co_counts = defaultdict(Counter)
    for e, f in tqdm(data):
        for e_word in e:
            for f_word in f:
                if uniform:
                    co_counts[e_word][f_word] = 1
                else:
                    co_counts[e_word][f_word] = random()
    for e_word in co_counts:
        normalization_factor = sum(list(co_counts[e_word].values()))
        for f_word in co_counts[e_word]:
            co_counts[e_word][f_word] = co_counts[e_word][f_word] / normalization_factor
    return co_counts

def EM_IBM1(data, validation, max_steps=20, translate_dict=None, epochs_trained=0):
    print("Initializing translation dictionary.")
    if translate_dict is None:
        translate_dict = initialize_t(data)
    for iteration in range(epochs_trained, epochs_trained + max_steps):
        change = False
        fname = 'iteration' + str(iteration) + '.txt'
        counts = Counter()
        co_counts = defaultdict(Counter)
        
        print("Expectation step {}".format(iteration + 1))
        for e_s, f_s in tqdm(data):
            for f in f_s:
                sum_of_probs = sum([translate_dict[e2][f] for e2 in e_s])
                for e in e_s:
                    delta = translate_dict[e][f] / sum_of_probs
                    co_counts[e][f] += delta
                    counts[e] += delta

        print("Maximisation step {}".format(iteration + 1))
        for e in co_counts:
            for f in co_counts[e]:
                new_value = co_counts[e][f] / counts[e]
                if abs(translate_dict[e][f] - new_value) > 1e-5:
                    change = True
                translate_dict[e][f] = new_value
        if not change:
            break
        #writing the iteration files in naacl for AER use
        alignments = align_all(validation, translate_dict, fname)
        ll = log_likelihood(data, translate_dict)
        aer = test("", alignments)
        print("Log likelihood: {}, AER: {}".format(ll, aer))
        pickle.dump(translate_dict, open("translate_dicts/ibm1_em_epoch_{}.pickle".format(iteration + 1), 'wb'))
    return translate_dict

print(type(training_data))

<class 'list'>


In [16]:


translate_dict = EM_IBM1(training_data[:10000], validation_data, 20)

Initializing translation dictionary.


100%|██████████████████████████████████| 10000/10000 [00:02<00:00, 3692.77it/s]


Expectation step 1


100%|███████████████████████████████████| 10000/10000 [00:11<00:00, 848.82it/s]


Maximisation step 1
Log likelihood: -704850.4815791112, AER: 0.46750727449078566
Expectation step 2


...

Maximisation step 2
Log likelihood: -524865.3773616902, AER: 0.3807205452775073
Expectation step 3


100%|███████████████████████████████████| 10000/10000 [00:11<00:00, 889.92it/s]


Maximisation step 3
Log likelihood: -429466.5232563358, AER: 0.36549707602339176
Expectation step 4


 36%|████████████▉                       | 3611/10000 [00:04<00:08, 751.82it/s]


KeyboardInterrupt: 

In [ ]:
def print_translation_probs(english_word, transdict):
    results = []
    for french_word in translate_dict[english_word]:
        results.append((translate_dict[english_word][french_word], french_word))
    results.sort(reverse=True)
    for r in results[:20]:
        print(r)
        
print_translation_probs('commissioners', translate_dict)

# Bayesian IBM1

In [70]:
from scipy.special import digamma, loggamma, gammaln
import math

def elbo(data, t, f_vocab, alpha, lambdas):
    ll = log_likelihood(data, t)
    print(ll)
    elbo = ll
    gammaln_alpha = gammaln(alpha)
    c = gammaln(alpha * len(f_vocab))
    for e in tqdm(t):
        a = sum([math.log(t[e][f]) * (alpha - lambdas[e][f])
                 +  gammaln(lambdas[e][f]) - gammaln_alpha for f in lambdas[e] if f != "-REST-"])
        b = gammaln(sum([(lambdas[e][f] if f in lambdas[e] else alpha) for f in f_vocab]))
        elbo += a - b + c
    return elbo

def VB_align(english_words, french_words, translate_dict, f_vocab, add_null=True):
    """Align one sentence pair, either with or without the NULL alignments.
    
    Args:
        english_words: list of english words
        french_words: list of french words
        translate_dict: dictionary with translation probabilities e to f
        add_null: boolean to indicate whether NULL alignments should be included

    Return:
        list of tuples
    """
    alignment = []
    for j, fword in enumerate(french_words):
        prior = 0.0
        alignment_j = 0
        for i, eword in enumerate(english_words):
            # Only include terms that are in the dictionary
            if eword in translate_dict:
                if fword in translate_dict[eword]:
                    prob = translate_dict[eword][fword]
                else:
                    prob = translate_dict[eword]["-REST-"]
                if prob > prior:
                    prior = prob
                    alignment_j = i
                # Add dependent on whether it's a NULL alignments
        if alignment_j != 0 or add_null:
            alignment.append((alignment_j, j + 1))
    return alignment

def VB_align_all(data, translate_dict, f_vocab, fname=None):
    """Create alignments for pairs of English and French sentences.
    Both save them as sets per sentence and pair and save to file.
    
    Args:
        validation: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f
        fname: filename to save alignments in, in NAACL format

    Returns:
        list of sets
    """
    file = open(fname, 'w')
    alignments = []
    for k, (english_words, french_words) in enumerate(data):
        alignment = VB_align(english_words, french_words, translate_dict, f_vocab, False)
        for pos1, pos2 in alignment:
            file.write("{} {} {}\n".format(str(k+1), str(pos1), str(pos2)))
        alignments.append(set(alignment))
    return alignments

def VB_IBM1(data, validation, alpha, max_steps=20, translate_dict=None):
    print("Initializing translation dictionary.")
    if translate_dict is None:
        translate_dict = initialize_t(data)
    e_vocab = translate_dict.keys()
    f_vocab = {f for e in translate_dict for f in translate_dict[e]}
    for iteration in range(max_steps):
        change = False
        fname = 'iteration' + str(iteration) + '.txt'
        lambdas = defaultdict(lambda : defaultdict(lambda : 0.05))

        print("Expectation step {}".format(iteration + 1))
        for e_s, f_s in tqdm(data):
            for f in f_s:
                sum_of_probs = sum([translate_dict[e2][f] for e2 in e_s])
                for e in e_s:
                    lambdas[e][f] += translate_dict[e][f] / sum_of_probs

        print("Maximisation step {}".format(iteration + 1))
        for e in tqdm(e_vocab):
            summation = 0
            for f2 in f_vocab:
                if f2 in lambdas[e]:
                    summation += lambdas[e][f2]
                else:
                    summation += alpha
            summation = digamma(summation)
            for f in translate_dict[e]:
                translate_dict[e][f] = np.exp(digamma(lambdas[e][f]) - summation)
            translate_dict[e]["-REST-"] = np.exp(digamma(alpha) - summation)
#                 if abs(translate_dict[e][f] - new_value) > 1e-5:
#                     change = True


#         if not change:
#             print("The translation probabilities did not change, so the model converged.")
#             break

        alignments = VB_align_all(validation, translate_dict, f_vocab, fname)
        #eb = elbo(data, translate_dict, f_vocab, alpha, lambdas)
        #print("Elbo: {}".format(eb))
        aer = test("", alignments)
        print("AER: {}".format(aer))
        pickle.dump(translate_dict, open("translate_dicts/ibm1_vb_epoch_{}.pickle".format(iteration + 1), 'wb'))
    return translate_dict

In [ ]:
alpha = 0.001
digamma_alpha = digamma(alpha)
translate_dict = VB_IBM1(training_data[:10000], validation_data, alpha, 15)

Initializing translation dictionary.


100%|██████████████████████████████████| 10000/10000 [00:02<00:00, 4163.20it/s]


Expectation step 1


100%|██████████████████████████████████| 10000/10000 [00:09<00:00, 1045.41it/s]


Maximisation step 1


...

100%|█████████████████████████████████████| 9156/9156 [00:37<00:00, 246.89it/s]


AER: 0.38099902056807056
Expectation step 3


100%|███████████████████████████████████| 10000/10000 [00:10<00:00, 940.91it/s]


Maximisation step 3


  1%|▍                                      | 90/9156 [00:00<00:41, 218.98it/s]

In [42]:
f_vocab = {f for e in translate_dict for f in translate_dict[e]}
s = 0
for word in f_vocab:
    if word == "-REST-":
        continue
    if word in translate_dict["commissioner"]:
        s += translate_dict["commissioner"][word]
#     else:
#         s += translate_dict['commissioner']["-REST-"]
print(s)

0.174540346609


In [58]:
#t = pickle.load(open("translate_dicts/ibm1_vb_epoch_1.pickle", 'rb'))
alignments = VB_align_all(validation_data, t, f_vocab, "test.txt")
aer = test("", alignments)
print(aer)

0.34416826003824097


In [68]:
t = translate_dict
for e in tqdm(t):
    in_t = set(t[e].keys())
    summation = sum(list(t[e].values()))
    summation += t[e]["-REST-"] * (len(f_vocab - in_t) - 1)
    for f in t[e]:
        t[e][f] = t[e][f] / summation

100%|███████████████████████████████████| 25593/25593 [02:16<00:00, 188.17it/s]


In [69]:
alignments = VB_align_all(validation_data, t, f_vocab, "test.txt")
aer = test("", alignments)
print(aer)

0.418957345971564


In [62]:
for e in tqdm(t):
    in_t = set(t[e].keys())
    summation = sum(list(t[e].values()))
    #summation += t[e]["-REST-"] * (len(f_vocab - in_t) - 1)
    print(summation)

  0%|                                                | 0/25593 [00:00<?, ?it/s]

0.999999497641
0.999997716097
0.999999281381
0.999999411297
0.999999126301
0.999997387859
0.999999895507
0.999999906671
0.999998895044
0.999998829886
0.999999550841
0.999998962425
0.999998080834
0.999998456589
0.999999801545
0.999999807165
0.999998587863
0.999999191608
0.999999721121
0.999998648331
0.999999873051
0.999999628536
0.999999381312


  0%|                                      | 23/25593 [00:00<01:52, 227.72it/s]

0.999999917308
0.999999584334
0.999999832483
0.999997968067
0.999994936926
0.999998652631
0.999998599311
0.999999558452
0.99999888965
0.999999454007
0.999999831311
0.999996520068
0.99999890755
0.999999111168
0.999999525426
0.999999394342
0.999998763201
0.99999968509
0.999997593802
0.999998789898
0.99999960183
0.999998816103
0.999999276782
0.999999371983


  0%|                                      | 47/25593 [00:00<01:50, 231.52it/s]

...

  3%|█                                    | 711/25593 [00:03<01:54, 217.83it/s]

0.999999165698
0.999999901292
0.999999825321
0.999998491562
0.99999975535
0.999997309093
0.999999061241
0.999994731514
0.999998868125
0.999999735588
0.999999838394
0.999999139423
0.999992879029
0.999994142946
0.999999697357
0.999999750511
0.999999372278
0.999996074686
0.999995713535
0.999998997714
0.999999267094
0.999996821708
0.999999431982
0.999995392311
0.999998534273


  3%|█                                    | 736/25593 [00:03<01:53, 218.72it/s]

0.999998577012
0.9999949753
0.999998975222
0.99999953254
0.999999637869
0.999998343767
0.999999829004
0.99999952972
0.999999505134
0.999995748763
0.999999797661
0.99999986772
0.999999441636
0.999999592096


KeyboardInterrupt: 